<a href="https://colab.research.google.com/github/Lopes258/LUCAS_LOPES_DDF_TECH_012026/blob/main/DADOSFERA_PROJETO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BASE DE DADOS BRAZILIAN E-COMERCE by OLIST

#GLOSSARIO
1.   Gerenciamento de Dependências
2.   Baixar e preparar os dados
3.   ML para a uma nova tabela sobre reviews

## GERENCIAMENTO DE DEPENDÊNCIAS

In [71]:
!pip install kaggle kagglehub
!pip install unidecode
!pip install pyodbc sqlalchemy




In [72]:
import kagglehub
import os

import pandas as pd
import numpy as np
import unicodedata

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## BASE DE DADOS OLIST DATABASE

In [73]:
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
print(path)



Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
/kaggle/input/brazilian-ecommerce


In [74]:
os.listdir(path)

['olist_customers_dataset.csv',
 'olist_sellers_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_products_dataset.csv',
 'olist_geolocation_dataset.csv',
 'product_category_name_translation.csv',
 'olist_orders_dataset.csv',
 'olist_order_payments_dataset.csv']

In [75]:
files = {
    "orders": "olist_orders_dataset.csv",
    "items": "olist_order_items_dataset.csv",
    "reviews": "olist_order_reviews_dataset.csv",
    "products": "olist_products_dataset.csv",
    "customers": "olist_customers_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "payment": "olist_order_payments_dataset.csv"
}

dfs = {}

for nome, arquivo in files.items():
    dfs[nome] = pd.read_csv(f"{path}/{arquivo}")
    print(nome, dfs[nome].shape)


orders (99441, 8)
items (112650, 7)
reviews (99224, 7)
products (32951, 9)
customers (99441, 5)
geolocation (1000163, 5)
sellers (3095, 4)
payment (103886, 5)


In [76]:
df_orders = dfs["orders"]
df_items = dfs["items"]
df_reviews = dfs["reviews"]
df_products = dfs["products"]
df_customers = dfs["customers"]
df_geolocation = dfs["geolocation"]
df_sellers = dfs["sellers"]
df_payment = dfs["payment"]


In [77]:
df_payment.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [78]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [79]:
df_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [80]:
df_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [81]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [82]:
def padronizar_texto(texto):
    if pd.isna(texto):
        return "desconhecido"
    texto = texto.lower()
    texto = unicodedata.normalize("NFD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.strip()


def padronizar_data(serie):
    return pd.to_datetime(serie, errors="coerce")


def padronizar_numerico(serie):
    return pd.to_numeric(serie, errors="coerce").fillna(0)


In [83]:
def tratar_dataframe(df):
    df = df.copy()

    # Remover duplicados
    df.drop_duplicates(inplace=True)

    for col in df.columns:
        # Texto
        if df[col].dtype == "object":
            df[col] = df[col].apply(padronizar_texto)

        # Datas
        elif "date" in col or "timestamp" in col:
            df[col] = padronizar_data(df[col])

        # Numéricos
        elif pd.api.types.is_numeric_dtype(df[col]):
            df[col] = padronizar_numerico(df[col])

    return df


In [84]:
tabelas_raw = {
    "orders": df_orders,
    "payments": df_payment,
    "customers": df_customers,
    "reviews": df_reviews,
    "items": df_items,
    "products": df_products,
    "geolocation": df_geolocation,
    "sellers": df_sellers
}

tabelas_trusted = {}

for nome, df in tabelas_raw.items():
    tabelas_trusted[nome] = tratar_dataframe(df)
    print(f"✅ {nome} tratado")


✅ orders tratado
✅ payments tratado
✅ customers tratado
✅ reviews tratado
✅ items tratado
✅ products tratado
✅ geolocation tratado
✅ sellers tratado


In [85]:
for nome, df in tabelas_raw.items():
    print(f"\nRAW - {nome}")
    print(df.isna().sum().head())

for nome, df in tabelas_trusted.items():
    print(f"\nTRUSTED - {nome}")
    print(df.isna().sum().head())



RAW - orders
order_id                      0
customer_id                   0
order_status                  0
order_purchase_timestamp      0
order_approved_at           160
dtype: int64

RAW - payments
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

RAW - customers
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

RAW - reviews
review_id                     0
order_id                      0
review_score                  0
review_comment_title      87656
review_comment_message    58247
dtype: int64

RAW - items
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
dtype: int64

RAW - products
product_id                      0
product_category_name         610
product_name_lenght           610
product_descripti

Criar uma tabela de historico de pagamentos para cada vendedor e tambem uma para os consumidores por dia mas colocando uma coluna para a "ANO-SEMANA"

In [86]:
payments_agg = (
    df_payment
    .groupby("order_id", as_index=False)
    .agg({
        "payment_value": "sum",
        "payment_type": "first"
    })
)


In [87]:
df = df_orders.merge(
    payments_agg,
    on="order_id",
    how="left"
)


In [88]:
df = df.merge(
    df_customers[[
        "customer_id",
        "customer_city",
        "customer_state"
    ]],
    on="customer_id",
    how="left"
)


In [89]:
df["order_purchase_timestamp"] = pd.to_datetime(
    df["order_purchase_timestamp"],
    errors="coerce"
)

df["year_week"] = (
    df["order_purchase_timestamp"].dt.year.astype(str)
    + "-"
    + df["order_purchase_timestamp"].dt.isocalendar().week.astype(str)
)

df["month"] = df["order_purchase_timestamp"].dt.month
df["day"] = df["order_purchase_timestamp"].dt.day
df["hour"] = df["order_purchase_timestamp"].dt.hour



In [90]:
def periodo_dia(hour):
    if 0 <= hour < 6:
        return "madrugada"
    elif 6 <= hour < 12:
        return "manha"
    elif 12 <= hour < 18:
        return "tarde"
    else:
        return "noite"

df["periodo_dia"] = df["hour"].apply(periodo_dia)


In [91]:
historico_compras = df[[
    "order_id",
    "customer_id",
    "customer_state",
    "customer_city",
    "payment_type",
    "payment_value",
    "order_purchase_timestamp",
    "year_week",
    "month",
    "day",
    "hour",
    "periodo_dia"
]]


In [92]:
historico_compras.head()

,order_id,customer_id,customer_state,customer_city,payment_type,payment_value,order_purchase_timestamp,year_week,month,day,hour,periodo_dia
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,SP,sao paulo,credit_card,38.71,2017-10-02 10:56:33,2017-40,10,2,10,manha
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,BA,barreiras,boleto,141.46,2018-07-24 20:41:37,2018-30,7,24,20,noite
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,GO,vianopolis,credit_card,179.12,2018-08-08 08:38:49,2018-32,8,8,8,manha
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,RN,sao goncalo do amarante,credit_card,72.20,2017-11-18 19:28:06,2017-46,11,18,19,noite
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,SP,santo andre,credit_card,28.62,2018-02-13 21:18:39,2018-7,2,13,21,noite


In [93]:
output_path = "/content/data/csv"
os.makedirs(output_path, exist_ok=True)

In [94]:
for nome, df in tabelas_raw.items():
    caminho = f"{output_path}/{nome}_raw.csv"
    df.to_csv(caminho, index=False)
    print(f"Arquivo salvo: {caminho}")

Arquivo salvo: /content/data/csv/orders_raw.csv
Arquivo salvo: /content/data/csv/payments_raw.csv
Arquivo salvo: /content/data/csv/customers_raw.csv
Arquivo salvo: /content/data/csv/reviews_raw.csv
Arquivo salvo: /content/data/csv/items_raw.csv
Arquivo salvo: /content/data/csv/products_raw.csv
Arquivo salvo: /content/data/csv/geolocation_raw.csv
Arquivo salvo: /content/data/csv/sellers_raw.csv


In [95]:

for nome, df in tabelas_trusted.items():
    caminho = f"{output_path}/{nome}_trusted.csv"
    df.to_csv(caminho, index=False)
    print(f"Arquivo salvo: {caminho}")

Arquivo salvo: /content/data/csv/orders_trusted.csv
Arquivo salvo: /content/data/csv/payments_trusted.csv
Arquivo salvo: /content/data/csv/customers_trusted.csv
Arquivo salvo: /content/data/csv/reviews_trusted.csv
Arquivo salvo: /content/data/csv/items_trusted.csv
Arquivo salvo: /content/data/csv/products_trusted.csv
Arquivo salvo: /content/data/csv/geolocation_trusted.csv
Arquivo salvo: /content/data/csv/sellers_trusted.csv


## MACHINE LEARNING

In [96]:
df_reviews = dfs["reviews"]

df_reviews.head()


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [97]:
df = df_reviews

df = df[["review_comment_message", "review_score"]]
df.dropna(subset=["review_comment_message"], inplace=True)



/tmp/ipython-input-1720403419.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["review_comment_message"], inplace=True)


In [98]:
def criar_sentimento(score):
    if score <= 2:
        return "negativo"
    elif score == 3:
        return "neutro"
    else:
        return "positivo"

df["sentimento"] = df["review_score"].apply(criar_sentimento)




/tmp/ipython-input-2316040410.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentimento"] = df["review_score"].apply(criar_sentimento)


In [99]:
stopwords_pt = stopwords.words('portuguese')

# NÃO remover palavras de negação
stopwords_pt = [w for w in stopwords_pt if w not in ["não", "nunca", "jamais"]]

def limpar_texto(texto):
    texto = texto.lower()
    texto = unidecode(texto)
    return texto

df["review_limpo"] = df["review_comment_message"].apply(limpar_texto)


/tmp/ipython-input-4244909451.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review_limpo"] = df["review_comment_message"].apply(limpar_texto)


In [100]:
X = df["review_limpo"]
y = df["sentimento"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [101]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words=stopwords_pt,
        max_features=10000,
        ngram_range=(1, 2),  # unigram + bigram
        min_df=5
    )),
    ("model", LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    ))
])


In [102]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=10000, min_df=5,
                                 ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('model', LogisticRegression(max_iter=1000, n_jobs=-1))])

In [103]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

    negativo       0.78      0.87      0.82      2178
      neutro       0.41      0.08      0.13       712
    positivo       0.89      0.95      0.92      5306

    accuracy                           0.85      8196
   macro avg       0.70      0.63      0.62      8196
weighted avg       0.82      0.85      0.83      8196

[[1892   39  247]
 [ 308   56  348]
 [ 228   40 5038]]


In [104]:
pipeline.predict([
    "entrega atrasada produto veio quebrado",
    "produto excelente chegou antes do prazo",
    "produto ok nada demais",
    "espera mais do produto"
])


array(['negativo', 'positivo', 'positivo', 'negativo'], dtype=object)

In [105]:
df["review_limpo"] = df["review_comment_message"].fillna("").apply(limpar_texto)

df["sentimento_modelo"] = pipeline.predict(df["review_limpo"])


/tmp/ipython-input-1333552226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review_limpo"] = df["review_comment_message"].fillna("").apply(limpar_texto)
/tmp/ipython-input-1333552226.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentimento_modelo"] = pipeline.predict(df["review_limpo"])


In [106]:
df[[
    "review_comment_message",
    "review_score",
    "sentimento_modelo"
]].head(10)


,review_comment_message,review_score,sentimento_modelo
3,Recebi bem antes do prazo estipulado.,5,positivo
4,Parabéns lojas lannister adorei comprar pela I...,5,positivo
9,aparelho eficiente. no site a marca do aparelh...,4,positivo
12,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",4,positivo
15,"Vendedor confiável, produto ok e entrega antes...",5,positivo
16,"GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E...",2,negativo
19,Péssimo,1,negativo
22,Loja nota 10,5,positivo
24,obrigado pela atençao amim dispensada,5,positivo
27,A compra foi realizada facilmente.\r\nA entreg...,5,positivo


In [107]:
tabelas_refined = {}
reviews_sentimento = df[[
    "review_comment_message",
    "review_score",
    "sentimento_modelo"
]]


In [108]:
tabelas_refined['historico_compras'] = historico_compras
tabelas_refined['reviews_sentimento'] = reviews_sentimento


In [109]:
for nome, df in tabelas_refined.items():
    caminho = f"{output_path}/{nome}.csv"
    df.to_csv(caminho, index=False)
    print(f"Arquivo salvo: {caminho}")

Arquivo salvo: /content/data/csv/historico_compras.csv
Arquivo salvo: /content/data/csv/reviews_sentimento.csv
